## AiCore Multinational Retail Data Centralisation Project

Milestone 4: Query the data using QL

Author: Kristina Gorkovskaya

Date: 2023-05-11

In [40]:
# Import libraries and connect to database

import pandas as pd
from database_utils import DatabaseConnector
from sqlalchemy import text

db_connector = DatabaseConnector("db_creds_sales_data.yaml")
db_connector.init_db_engine(autocommit=True)
engine = db_connector.engine


def execute(sql):
    '''Helper function for executing SQL and printing results.

    Arguments:
        sql (string): SQL query
    
    Returns:
        None'''
        
    with engine.connect() as con:
	    df = pd.read_sql_query(sql=text(sql), con=con)
    print(df)


Loading database credentials from file: db_creds_sales_data.yaml
Connecting to database: sales_data


### 1. How many stores does the business have and in which countries?

In [41]:
sql = '''
SELECT country_code, count(*) AS total_no_stores
FROM dim_store_details
GROUP BY country_code
ORDER BY country_code;
'''

execute(sql)

  country_code  total_no_stores
0           DE              141
1           GB              266
2           US               34


### 2. Which locations currently have the most stores?

In [42]:
sql = '''
SELECT locality, count(*) AS total_no_stores
FROM dim_store_details
GROUP BY locality
ORDER BY total_no_stores DESC
LIMIT 10;
'''

execute(sql)

       locality  total_no_stores
0    Chapletown               14
1        Belper               13
2        Bushey               12
3        Exeter               11
4  High Wycombe               10
5    Rutherglen               10
6      Arbroath               10
7      Surbiton                9
8       Lancing                9
9      Aberdeen                9


### 3. Which months produce the highest average cost of sales?

In [43]:
sql = '''
SELECT
	ROUND(CAST(SUM(spend) AS numeric), 2) AS total_sales,
	month
FROM (
	SELECT 
		ord.product_quantity * prod.product_price AS spend,
		dt.month
	FROM orders_table ord
	INNER JOIN dim_date_times dt
		ON ord.date_uuid = dt.date_uuid
	INNER JOIN dim_products prod
		ON ord.product_code = prod.product_code
) x
GROUP BY month
ORDER BY total_sales DESC;
'''

execute(sql)

    total_sales month
0     673295.68     8
1     668041.45     1
2     657335.84    10
3     650321.43     5
4     645741.70     7
5     645463.00     3
6     635578.99     6
7     635329.09    12
8     633993.62     9
9     630757.08    11
10    630022.77     4
11    616452.99     2


### 4. How many sales are coming from online?

In [44]:
sql = '''
SELECT
	COUNT(*) AS numbers_of_sales,
	SUM(product_quantity) AS product_quantity_count,
	location
FROM (
	SELECT 
		ord.product_quantity,
		CASE
			WHEN st.store_type = 'Web Portal' THEN 'Web'
			ELSE 'Offline'
		END AS location
	FROM orders_table ord
	INNER JOIN dim_store_details st
		ON ord.store_code = st.store_code
) x
GROUP BY location
ORDER BY location DESC;
''' 

execute(sql)

   numbers_of_sales  product_quantity_count location
0             26957                  107739      Web
1             93166                  374047  Offline


### 5. What percentage of sales come through each type of store?

In [45]:
sql = ''' 
SELECT
	store_type,
	ROUND(CAST(SUM(sale) AS NUMERIC), 2) AS total_sales,
	ROUND(CAST(100 * SUM(sale) / total AS NUMERIC), 2) AS "percentage_total(%)"
FROM (
	SELECT 
		st.store_type,
		ord.product_quantity * prod.product_price AS sale
	FROM orders_table ord
	INNER JOIN dim_store_details st
		ON ord.store_code = st.store_code
	INNER JOIN dim_products prod
		ON ord.product_code = prod.product_code
) x
CROSS JOIN (
	SELECT SUM(ord.product_quantity * prod.product_price) AS total
	FROM orders_table ord
	INNER JOIN dim_products prod
	ON ord.product_code = prod.product_code
) y
GROUP BY x.store_type, y.total
ORDER BY total_sales DESC;
''' 

execute(sql)

    store_type  total_sales  percentage_total(%)
0        Local   3440896.52                44.56
1   Web Portal   1726547.05                22.36
2  Super Store   1224293.65                15.85
3   Mall Kiosk    698791.61                 9.05
4       Outlet    631804.81                 8.18


### 6. Which months produced the highest sales?

In [46]:
sql = ''' 
SELECT
	ROUND(CAST(SUM(sale) AS NUMERIC), 2) AS total_sales,
	year,
	month
FROM (
	SELECT 
		dt.year,
		dt.month,
		ord.product_quantity * prod.product_price AS sale
	FROM orders_table ord
	INNER JOIN dim_products prod
		ON ord.product_code = prod.product_code
	INNER JOIN dim_date_times dt
		ON ord.date_uuid = dt.date_uuid
) x
GROUP BY year, month
ORDER BY total_sales DESC
LIMIT 10;
''' 

execute(sql)

   total_sales  year month
0     27936.77  1994     3
1     27356.14  2019     1
2     27091.67  2009     8
3     26679.98  1997    11
4     26310.97  2018    12
5     26277.72  2019     8
6     26236.67  2017     9
7     25798.12  2010     5
8     25648.29  1996     8
9     25614.54  2000     1


### 7. What is our staff headcount?

In [47]:
sql = ''' 
SELECT 
	SUM(staff_numbers) AS total_staff_numbers,
	country_code
FROM dim_store_details
GROUP BY country_code
ORDER BY total_staff_numbers DESC;
'''

execute(sql)

   total_staff_numbers country_code
0                13132           GB
1                 6054           DE
2                 1304           US


### 8. Which German store type is selling the most?

In [ ]:
sql = ''' 
SELECT 
	ROUND(CAST(SUM(sale) AS NUMERIC), 2) AS total_sales,
	store_type,
	country_code
FROM (
SELECT
	st.store_type,
	st.country_code,
	ord.product_quantity * prod.product_price AS sale
FROM orders_table ord
INNER JOIN (
	SELECT store_code, store_type, country_code
	FROM dim_store_details 
	WHERE country_code = 'DE'
	) st
	ON ord.store_code = st.store_code
INNER JOIN dim_products prod
	ON ord.product_code = prod.product_code
) x
GROUP BY 
	store_type,
	country_code
ORDER BY 
	total_sales;
''' 

execute(sql)